## Grouping and Aggregating

### Aggregate Functions - these operators work on a set of values from mulyiple rows at once

- COUNT - counts the number of non-NULL values in a column,works on BIT column, no other does.
- SUM 
- AVG 
- MIN - can be used on character data as well as date and numeric
- MAX

AGGREGATE functions dont work on TEST,NTEST and IMAGE columns  
<span style="color: var(--vscode-foreground);">except for COUNT others dont work on special data types like hierarchyID</span>

COUNT can be used with \* to give the count of all the rows  
  
all the aggregate functions ignore null except for count when using with \*

  

SYNTAX

> SELECT \<AGG FUNC\>(\<COL1\>)  
> FROM \<TABLE\>

In [ ]:
SELECT COUNT(*) AS COUNTOFROWS,
    MAX(H.TotalDue) AS MaxTotal,
    MIN(H.TotalDue) AS MINTOTAL,
    SUM(H.TotalDue) AS SUMOFTOTAL,
    AVG(H.TotalDue) AS AVGTOTAL 
FROM Sales.SalesOrderHeader AS H;

In [ ]:
SELECT MIN(Name) AS MinName,
 MAX(Name) AS MaxName,
 MIN(SellStartDate) AS MinSellStartDate
FROM Production.Product;

In [ ]:
SELECT COUNT(*) AS CountOfRows,
 COUNT(Color) AS CountOfColor,
 COUNT(DISTINCT Color) AS CountOfDistinctColor
FROM Production.Product;

We can also have simple columns along wth aggregate functions as part of the same column.

### Exercise 7.1

Use the AdventureWorks2019 database to complete Questions 1 to 5. Switch to the WideWorldImporters database to complete this exercise.

1\. Write a query to determine the number of customers in the Sales.Customer table.

In [ ]:
SELECT COUNT(DISTINCT C.CustomerID)
FROM Sales.Customer AS C;

2\. Write a query that retrieves the total number of products ordered. Use the OrderQty column of the Sales.SalesOrderDetail table and the SUM function.

In [ ]:
SELECT SUM(OrderQty)
FROM Sales.SalesOrderDetail

3\. Write a query to determine the price of the most expensive product ordered. Use the UnitPrice column of the Sales.SalesOrderDetail table.

In [ ]:
SELECT MAX(D.UnitPrice)
FROM Sales.SalesOrderDetail AS D;

4\. Write a query to determine the average freight amount in the Sales. SalesOrderHeader table.

In [ ]:
SELECT AVG(H.Freight)
FROM Sales.SalesOrderHeader AS H;

5\. Write a query using the Production.Product table that displays the minimum, maximum, and average ListPrice.

In [ ]:
SELECT MIN(ListPrice),MAX(ListPrice),AVG(ListPrice) 
FROM Production.Product;

### GROUP BY

use when we want aggreagation on the basis of some common value, use with categorical data

### Grouping on columns

- for when we wanto calculate the sum of all the orders grouped by the date or grouped by the customer
- An additional column may be included in the select list, the column being the one we group the data on
- once you have included normal non aggregate columns in the select list we must group by on them, or you will get an error

SYNTAX

> SELECT \<AGG FUNC\>(\<COL1\>), \<COL2\>
> 
> FROM \<TABLE\> 
> 
> GROUP BY \<COL2\>

In [ ]:
SELECT CustomerID,SUM(TotalDue) AS TotalPerCustomer
FROM Sales.SalesOrderHeader as H 
GROUP BY CustomerID;

In [ ]:
SELECT TerritoryID,AVG(TotalDue) AS AveragePerTerritory
FROM Sales.SalesOrderHeader
GROUP BY TerritoryID;

### GROUP ON EXPRESSIONS

- we can group on soem expression applied on some column as well

In [ ]:
SELECT COUNT(*) AS CountOfOrders, YEAR(OrderDate) AS OrderYear
FROM Sales.SalesOrderHeader
GROUP BY YEAR(OrderDate);

- you must group by on the same expressio in the selelct list else the result will ont be what you expected

EXERCISE 7.2

Use the AdventureWorks2019 database to complete Questions 1 to 4. Switch to the WideWorldImporters database to complete the exercise.

1\. Write a query that shows the total number of items ordered for each product. Use the Sales.SalesOrderDetail table to write the query.

In [ ]:
SELECT COUNT(D.OrderQty),P.Name
FROM Sales.SalesOrderDetail AS D
INNER JOIN Production.Product AS P ON P.ProductID=D.ProductID
GROUP BY P.Name